#### Importar bibliotecas e funcionalidades

In [2]:
import os
import pandas as pd

#### Identificar colunas


In [ ]:
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

path = f"{parent_dir}\\excel_data"

cols_by_archive = {}
for archive in os.listdir(path):
    if archive[:5] == "clean":
        df = pd.read_excel(f"{path}\\{archive}")
        cols_by_archive[archive] = df.columns.to_list()
        
print(cols_by_archive)

#### Ler os arquivos(planilhas)


In [5]:
data_dir = path
df_fato_vendas = pd.read_excel(f"{data_dir}\\clean_fato_vendas.xlsx")
df_dim_vendedor = pd.read_excel(f"{data_dir}\\clean_dim_vendedor.xlsx")
df_dim_produtos = pd.read_excel(f"{data_dir}\\clean_dim_produtos.xlsx")
df_dim_familia_produtos = pd.read_excel(f"{data_dir}\\clean_dim_familia_produtos.xlsx")

#### Mesclar planilhas por colunas iguais


In [ ]:
df_timed_stats = pd.merge(df_fato_vendas, df_dim_produtos[["codigo_familia", "descricaoproduto", "codigo_produto"]], on="codigo_produto", how="inner")
display(df_timed_stats.head())

#### Mesclar planilhas por colunas iguais


In [ ]:
df_timed_stats = pd.merge(df_timed_stats, df_dim_familia_produtos[["descricaofamilia", "codigo_familia"]], on="codigo_familia", how="inner")
display(df_timed_stats)

#### Tranforamar tipos de dados


In [ ]:
df_timed_stats["data_venda"] = pd.to_datetime(df_timed_stats["data_venda"]).reset_index(drop=True)
display(df_timed_stats)

#### Separar "datetime" por ano e mes


In [ ]:
df_timed_stats["mes"] = df_timed_stats["data_venda"].dt.month
df_timed_stats["ano"] = df_timed_stats["data_venda"].dt.year
display(df_timed_stats)

#### Agrupar e mesclar colunas


In [ ]:
df_timed_stats_per_month = df_timed_stats.groupby(["codigo_familia", "mes", "ano"])["valor_monetario_total"].sum().reset_index()
df_timed_stats_per_month = pd.merge(df_timed_stats_per_month, df_dim_familia_produtos[["descricaofamilia", "codigo_familia"]], on="codigo_familia", how="inner")
display(df_timed_stats_per_month)

#### Salvar como nova planilha


In [12]:
df_timed_stats_per_month.to_excel(f"{data_dir}\\df_fam_prod_venda_timed.xlsx", index=False)